## Install huggingface transformers

In [1]:
!pip install transformers
!pip install pytorch-nlp


    100% |████████████████████████████████| 573kB 16.5MB/s ta 0:00:01
    100% |████████████████████████████████| 686kB 24.6MB/s ta 0:00:01
    100% |████████████████████████████████| 1.0MB 22.7MB/s ta 0:00:01
    100% |████████████████████████████████| 890kB 28.7MB/s ta 0:00:01
    100% |████████████████████████████████| 3.7MB 10.8MB/s ta 0:00:01
    100% |████████████████████████████████| 296kB 27.3MB/s ta 0:00:01
  Running setup.py bdist_wheel for sacremoses ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
fastai 1.0.60 requires nvidia-ml-py3, which is not installed.
You are using pip version 10.0.1, however version 20.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
    100% |████████████████████████████████| 92kB 13.8MB/s ta 0:00:01
fastai 1.0.60 requires nvidia-ml-py3, which is not installed.
You are using pip version 10.0.1, however 

In [2]:
import torch                                        # root package
from torch.utils.data import Dataset, DataLoader    # dataset representation and loading


In [3]:
import os
import boto3
import sagemaker
from sagemaker import get_execution_role

sess = sagemaker.Session()
role = get_execution_role()
print(role)

arn:aws:iam::733425554560:role/service-role/AmazonSageMaker-ExecutionRole-20200504T094270


## 1. Obtain dataset

### We don't want to use the prepared dataset as it, wo we compare the sample dataset into csv

In [4]:
import csv
from torchnlp.datasets import imdb_dataset
train, test = imdb_dataset(train=True,test=True)

with open('data/train.csv', 'w') as csvfile:
    csvwriter = csv.writer(csvfile, delimiter=',')
    csvwriter.writerow(['text','sentiment'])
    for i in train:
        csvwriter.writerow([i['text'],i['sentiment']])

with open('data/test.csv', 'w') as csvfile:
    csvwriter = csv.writer(csvfile, delimiter=',')
    csvwriter.writerow(['text','sentiment'])
    for i in test:
        csvwriter.writerow([i['text'],i['sentiment']])

In [5]:
import pandas as pd
train_df = pd.read_csv("data/train.csv")
test_df = pd.read_csv("data/test.csv")
mapping = {'neg': 0, 'pos': 1}
train_df = train_df.replace({'sentiment': mapping})
test_df = test_df.replace({'sentiment': mapping})

In [6]:
train_df.sample(10)

,text,sentiment
3460,"""Sasquatch Hunters"" actually wasn't as bad as ...",1
17539,"I don't understand how ""2 of us"" receive such ...",0
13966,Neat premise. Very unrealistic. What I learned...,0
9461,Does anyone else cry tears of joy when they wa...,1
19198,There is absolutely nothing in this movie that...,0
18,"From the mind of Robert Bloch, of ""Psycho"" fam...",1
10375,"I second the motion to make this into a movie,...",1
15459,I caught this stink bomb of a movie recently o...,0
9437,"Visconti's first feature, Ossessione is an ada...",1
22616,Nightmare Weekend stars a cast of ridiculous a...,0


In [7]:
test_df.sample(10)

,text,sentiment
9419,"Ben (a fine Charles Bateman), his young daught...",1
15019,This is only the fourth effort I’ve watched fr...,0
3862,While some of the things in Haggard are dumb a...,1
8740,"First, a word of caution. The DVD box describe...",1
9547,"Multiply named and strangely casted, ""One Dark...",1
993,I must admit that I didn't get around to seein...,1
24200,"I love comedies and I love independent films, ...",0
22979,"I'm usually quite tolerant of movies, and very...",0
8928,I remember this movie getting a lot of flak fr...,1
24856,The plot was not good.<br /><br />The special ...,0


In [8]:
train_df = train_df.sample(int(len(train_df)*0.1))
test_df = test_df.sample(int(len(test_df)*0.1))

In [9]:
train_sentences = train_df.text.values
train_labels = train_df.sentiment.values
test_sentences = test_df.text.values
test_labels = test_df.sentiment.values

In [10]:
import numpy as np
os.makedirs("./datasets/train", exist_ok=True)
np.save("./datasets/train/train_sentences.npy", train_sentences)
np.save("./datasets/train/train_labels.npy", train_labels)
os.makedirs("./datasets/test", exist_ok=True)
np.save("./datasets/test/test_sentences.npy", test_sentences)
np.save("./datasets/test/test_labels.npy", test_labels)


In [11]:
BUCKET_NAME = sess.default_bucket()
PREFIX = 'bert-classification-janossch'

traindata_s3_prefix = f"{PREFIX}/datasets/train"
testdata_s3_prefix = f"{PREFIX}/datasets/test"
output_s3 = f"s3://{BUCKET_NAME}/{PREFIX}/models/"

In [12]:
train_s3 = sess.upload_data(path="./datasets/train/", bucket=BUCKET_NAME, key_prefix=traindata_s3_prefix)
test_s3 = sess.upload_data(path="./datasets/test/", bucket=BUCKET_NAME, key_prefix=testdata_s3_prefix)

### Tokenize Dataset (moved to the training script)

In [13]:
train_channel = f"s3://{BUCKET_NAME}/{PREFIX}/datasets/train/"
test_channel = f"s3://{BUCKET_NAME}/{PREFIX}/datasets/test/"

In [25]:
from sagemaker.pytorch.estimator import PyTorch as PyTorchEstimator
estimator = PyTorchEstimator(
    entry_point="janossch-train.py",
    source_dir="src",
    
    base_job_name="bert-classification",
    output_path=f"s3://{BUCKET_NAME}/{PREFIX}/",
    
    framework_version="1.4.0",
    py_version="py3",
    
    role=role,
    train_instance_count=1,
    train_instance_type="local_gpu",
    train_max_run=60*60,
    train_max_wait=60*60,
    
    hyperparameters={
        "seed": 4711,
        "log_level": "DEBUG",
        'batch-size': 32,
    }
)

In [26]:
estimator.fit({ "train": train_channel, "test": test_channel })

Creating tmp8f6bj_il_algo-1-tnt5x_1 ... 
Attaching to tmp8f6bj_il_algo-1-tnt5x_12mdone
algo-1-tnt5x_1  | 2020-05-06 06:31:14,852 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
algo-1-tnt5x_1  | 2020-05-06 06:31:14,879 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
algo-1-tnt5x_1  | 2020-05-06 06:31:14,882 sagemaker_pytorch_container.training INFO     Invoking user training script.
algo-1-tnt5x_1  | 2020-05-06 06:31:15,002 sagemaker-containers INFO     Module default_user_module_name does not provide a setup.py. 
algo-1-tnt5x_1  | Generating setup.py
algo-1-tnt5x_1  | 2020-05-06 06:31:15,002 sagemaker-containers INFO     Generating setup.cfg
algo-1-tnt5x_1  | 2020-05-06 06:31:15,002 sagemaker-containers INFO     Generating MANIFEST.in
algo-1-tnt5x_1  | 2020-05-06 06:31:15,003 sagemaker-containers INFO     Installing module with the following command:
algo-1-tnt5x_1  | /opt/conda/bin/python -m pip install . -

algo-1-tnt5x_1  | There are 1 GPU(s) available.
algo-1-tnt5x_1  | We will use the GPU: Tesla K80
algo-1-tnt5x_1  | 
algo-1-tnt5x_1  | ======== Epoch 1 / 4 ========
algo-1-tnt5x_1  | Training...
algo-1-tnt5x_1  |   Batch    40  of     79.    Elapsed: 0:00:23.
algo-1-tnt5x_1  | 
algo-1-tnt5x_1  |   Average training loss: 0.59
algo-1-tnt5x_1  |   Training epoch took: 0:00:45
algo-1-tnt5x_1  | 
algo-1-tnt5x_1  | Running Validation...
algo-1-tnt5x_1  |   Accuracy: 0.74
algo-1-tnt5x_1  |   Validation Loss: 0.02
algo-1-tnt5x_1  |   Validation took: 0:00:14
algo-1-tnt5x_1  | 
algo-1-tnt5x_1  | ======== Epoch 2 / 4 ========
algo-1-tnt5x_1  | Training...
algo-1-tnt5x_1  |   Batch    40  of     79.    Elapsed: 0:00:23.
algo-1-tnt5x_1  | 
algo-1-tnt5x_1  |   Average training loss: 0.42
algo-1-tnt5x_1  |   Training epoch took: 0:00:45
algo-1-tnt5x_1  | 
algo-1-tnt5x_1  | Running Validation...
algo-1-tnt5x_1  |   Accuracy: 0.78
algo-1-tnt5x_1  |   Validation Loss: 0.01
algo-1-tnt5x_1  |   Validation